### 1) Import Libraries

In [143]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV 
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

#import xgboost as xgb
#import lightgbm as lgb

sns.set(style='white', context='notebook', palette='deep')

### 2) Import Data

In [144]:
#Train_Masked has extra columns: Delivery destination (day, month, time)

train_df = pd.read_csv("D:\Temp\Train.csv")
test_df = pd.read_csv("D:\Temp\Test.csv")
riders_df = pd.read_csv("D:\Temp\Riders.csv")

print(train_df.shape, test_df.shape, riders_df.shape)
train_df.head()

(21201, 29) (7068, 25) (960, 5)


,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,...,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,...,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,...,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,...,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,...,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


### 3) Data Manipulation

In [145]:
#Drop data not available in test, Pickup Time + label = Arrival times

train_df = train_df.drop(['Arrival at Destination - Day of Month', 'Arrival at Destination - Weekday (Mo = 1)', 'Arrival at Destination - Time'], axis=1)


#### Creating Full_df

In [146]:
#Create (full_df = train + test) ** caution (dont shuffle, avoid drop/adding rows)
#explore training, make (column) changes to full, later we use the border to separate
#Be careful of information leakage

border = train_df.shape[0]
test_df['Time from Pickup to Arrival'] = [np.nan]* test_df.shape[0]
full_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

train_df.shape, test_df.shape, full_df.shape

((21201, 26), (7068, 26), (28269, 26))

#### Renaming columns

In [147]:
#Renaming columns (shorten, remove space, standardize)
new_names = {"Order No": "Order_No", "User Id": "User_Id", "Vehicle Type": "Vehicle_Type",
    "Personal or Business": "Personal_Business", "Placement - Day of Month": "Pla_Mon",
    "Placement - Weekday (Mo = 1)": "Pla_Weekday", "Placement - Time": "Pla_Time", 
    "Confirmation - Day of Month":"Con_Day_Mon", "Confirmation - Weekday (Mo = 1)": "Con_Weekday","Confirmation - Time": "Con_Time", 
    "Arrival at Pickup - Day of Month": "Arr_Pic_Mon", "Arrival at Pickup - Weekday (Mo = 1)": "Arr_Pic_Weekday", 
                "Arrival at Pickup - Time": "Arr_Pic_Time", "Platform Type": "Platform_Type",
     "Pickup - Day of Month": "Pickup_Mon", "Pickup - Weekday (Mo = 1)": "Pickup_Weekday",           
    "Pickup - Time": "Pickup_Time",  "Distance (KM)": "Distance(km)",
    "Precipitation in millimeters": "Precipitation(mm)", "Pickup Lat": "Pickup_Lat", "Pickup Long": "Pickup_Lon", 
    "Destination Lat": "Destination_Lat", "Destination Long":"Destination_Lon", "Rider Id": "Rider_Id",
                            "Time from Pickup to Arrival": "Time_Pic_Arr"
                           }

full_df = full_df.rename(columns=new_names)
full_df.columns

Index(['Order_No', 'User_Id', 'Vehicle_Type', 'Platform_Type',
       'Personal_Business', 'Pla_Mon', 'Pla_Weekday', 'Pla_Time',
       'Con_Day_Mon', 'Con_Weekday', 'Con_Time', 'Arr_Pic_Mon',
       'Arr_Pic_Weekday', 'Arr_Pic_Time', 'Pickup_Mon', 'Pickup_Weekday',
       'Pickup_Time', 'Distance(km)', 'Temperature', 'Precipitation(mm)',
       'Pickup_Lat', 'Pickup_Lon', 'Destination_Lat', 'Destination_Lon',
       'Rider_Id', 'Time_Pic_Arr'],
      dtype='object')

#### Convert Time

In [148]:
#Convert Time from 12H to 24H

def convert_to_24hrs(fulldf):
    for col in fulldf.columns:
        if col.endswith("Time"):
            fulldf[col] = pd.to_datetime(fulldf[col], format='%I:%M:%S %p').dt.strftime("%H:%M:%S")
    return fulldf

full_df = convert_to_24hrs(full_df)

full_df[['Pla_Time', 'Con_Time' , 'Arr_Pic_Time', 'Pickup_Time']][3:6]


,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time
3,09:25:34,09:26:05,09:37:56,09:43:06
4,09:55:18,09:56:18,10:03:53,10:05:23
5,15:07:35,15:08:57,15:21:36,15:30:30


#### Filling Missing Values

In [149]:
#Filling Missing Values for temperatures and humidity

full_df['Temperature'] = full_df['Temperature'].fillna(full_df['Temperature'].mean())
full_df['Precipitation(mm)'].fillna(full_df['Precipitation(mm)'].mean(), inplace=True)

In [150]:
full_df.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Personal_Business,Pla_Mon,Pla_Weekday,Pla_Time,Con_Day_Mon,Con_Weekday,...,Pickup_Time,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Rider_Id,Time_Pic_Arr
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,09:35:46,9,5,...,10:27:30,4,20.400000,7.573502,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16,12,5,...,11:44:09,16,26.400000,7.573502,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993.0
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25,30,2,...,12:53:03,3,23.255689,7.573502,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455.0
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,09:25:34,15,5,...,09:43:06,9,19.200000,7.573502,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341.0
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,09:55:18,13,1,...,10:05:23,9,15.400000,7.573502,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214.0


#### Traversing Month and Weekday

In [151]:
#Since, we have not been given the actual dates & bikes (same day) were used, is Pick, Arrival date not the same?

month_cols = [col for col in full_df.columns if col.endswith("Mon")]
weekday_cols = [col for col in full_df.columns if col.endswith("Weekday")]

count = 0
instances_of_different_days = [];
for i, row in full_df.iterrows():
    if len(set(row[month_cols].values)) > 1:
        print(count+1, end="\r")
        count = count + 1
        instances_of_different_days.append(list(row[month_cols].values))
instances_of_different_days

[[17, 18, 18, 18], [11, 13, 13, 13]]

In [152]:
month_cols

['Pla_Mon', 'Con_Day_Mon', 'Arr_Pic_Mon', 'Pickup_Mon']

In [153]:
weekday_cols

['Pla_Weekday', 'Con_Weekday', 'Arr_Pic_Weekday', 'Pickup_Weekday']

#### Creating Month and Weekday columns

In [154]:
full_df['Day_of_Month'] = full_df[month_cols[0]]
full_df['Day_of_Week'] = full_df[weekday_cols[0]]

#### Dropping redundant columns

In [155]:
#All Vehicle types are Bikes, Vehicle Type is not necessary.
#Day & Weekday values are repeated in all rows except 2, we retain only one
full_df.drop(month_cols+weekday_cols, axis=1, inplace=True)
full_df.drop('Vehicle_Type', axis=1, inplace=True)

full_df.head(3)

,Order_No,User_Id,Platform_Type,Personal_Business,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Rider_Id,Time_Pic_Arr,Day_of_Month,Day_of_Week
0,Order_No_4211,User_Id_633,3,Business,09:35:46,09:40:10,10:04:47,10:27:30,4,20.400000,7.573502,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0,9,5
1,Order_No_25375,User_Id_2285,3,Personal,11:16:16,11:23:21,11:40:22,11:44:09,16,26.400000,7.573502,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993.0,12,5
2,Order_No_1899,User_Id_265,3,Business,12:39:25,12:42:44,12:49:34,12:53:03,3,23.255689,7.573502,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455.0,30,2


In [156]:
full_df.head()

,Order_No,User_Id,Platform_Type,Personal_Business,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Rider_Id,Time_Pic_Arr,Day_of_Month,Day_of_Week
0,Order_No_4211,User_Id_633,3,Business,09:35:46,09:40:10,10:04:47,10:27:30,4,20.400000,7.573502,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0,9,5
1,Order_No_25375,User_Id_2285,3,Personal,11:16:16,11:23:21,11:40:22,11:44:09,16,26.400000,7.573502,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993.0,12,5
2,Order_No_1899,User_Id_265,3,Business,12:39:25,12:42:44,12:49:34,12:53:03,3,23.255689,7.573502,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455.0,30,2
3,Order_No_9336,User_Id_1402,3,Business,09:25:34,09:26:05,09:37:56,09:43:06,9,19.200000,7.573502,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341.0,15,5
4,Order_No_27883,User_Id_1737,1,Personal,09:55:18,09:56:18,10:03:53,10:05:23,9,15.400000,7.573502,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214.0,13,1


In [157]:
full_df.columns

Index(['Order_No', 'User_Id', 'Platform_Type', 'Personal_Business', 'Pla_Time',
       'Con_Time', 'Arr_Pic_Time', 'Pickup_Time', 'Distance(km)',
       'Temperature', 'Precipitation(mm)', 'Pickup_Lat', 'Pickup_Lon',
       'Destination_Lat', 'Destination_Lon', 'Rider_Id', 'Time_Pic_Arr',
       'Day_of_Month', 'Day_of_Week'],
      dtype='object')

#### Variable Datatypes

In [158]:
numeric_cols = []
object_cols = []
time_cols = []
for k, v in full_df.dtypes.items():
    if (v != object):
        if (k != "Time_Pic_Arr"):
            numeric_cols.append(k)
    elif k.endswith("Time"):
        time_cols.append(k)
    else:
        object_cols.append(k)
full_df[numeric_cols].head(3) 

,Platform_Type,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Day_of_Month,Day_of_Week
0,3,4,20.400000,7.573502,-1.317755,36.830370,-1.300406,36.829741,9,5
1,3,16,26.400000,7.573502,-1.351453,36.899315,-1.295004,36.814358,12,5
2,3,3,23.255689,7.573502,-1.308284,36.843419,-1.300921,36.828195,30,2


In [159]:
full_df[time_cols].head(3)

,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time
0,09:35:46,09:40:10,10:04:47,10:27:30
1,11:16:16,11:23:21,11:40:22,11:44:09
2,12:39:25,12:42:44,12:49:34,12:53:03


In [160]:
full_df[object_cols].head(3)

,Order_No,User_Id,Personal_Business,Rider_Id
0,Order_No_4211,User_Id_633,Business,Rider_Id_432
1,Order_No_25375,User_Id_2285,Personal,Rider_Id_856
2,Order_No_1899,User_Id_265,Business,Rider_Id_155


#### Convert an object to numeric

In [161]:
#Convert an object to numeric (encoding)

le = LabelEncoder()
le.fit(full_df['Personal_Business'])
full_df['Personal_Business'] = le.transform(full_df['Personal_Business'])
full_df['Personal_Business'][:2]


0    0
1    1
Name: Personal_Business, dtype: int32

In [162]:
full_df.head()

,Order_No,User_Id,Platform_Type,Personal_Business,Pla_Time,Con_Time,Arr_Pic_Time,Pickup_Time,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Rider_Id,Time_Pic_Arr,Day_of_Month,Day_of_Week
0,Order_No_4211,User_Id_633,3,0,09:35:46,09:40:10,10:04:47,10:27:30,4,20.400000,7.573502,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745.0,9,5
1,Order_No_25375,User_Id_2285,3,1,11:16:16,11:23:21,11:40:22,11:44:09,16,26.400000,7.573502,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993.0,12,5
2,Order_No_1899,User_Id_265,3,0,12:39:25,12:42:44,12:49:34,12:53:03,3,23.255689,7.573502,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455.0,30,2
3,Order_No_9336,User_Id_1402,3,0,09:25:34,09:26:05,09:37:56,09:43:06,9,19.200000,7.573502,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341.0,15,5
4,Order_No_27883,User_Id_1737,1,1,09:55:18,09:56:18,10:03:53,10:05:23,9,15.400000,7.573502,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214.0,13,1


#### Feature Selection

In [163]:
features = numeric_cols + ['Personal_Business'] + ['Order_No']

data_df = full_df[features]

y = full_df[:border]['Time_Pic_Arr']
train = data_df[:border]
test = data_df[border:]

train.head()

,Platform_Type,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Day_of_Month,Day_of_Week,Personal_Business,Order_No
0,3,4,20.400000,7.573502,-1.317755,36.830370,-1.300406,36.829741,9,5,0,Order_No_4211
1,3,16,26.400000,7.573502,-1.351453,36.899315,-1.295004,36.814358,12,5,1,Order_No_25375
2,3,3,23.255689,7.573502,-1.308284,36.843419,-1.300921,36.828195,30,2,0,Order_No_1899
3,3,9,19.200000,7.573502,-1.281301,36.832396,-1.257147,36.795063,15,5,0,Order_No_9336
4,1,9,15.400000,7.573502,-1.266597,36.792118,-1.295041,36.809817,13,1,1,Order_No_27883


In [164]:
print(full_df.shape,data_df.shape,train.shape,test.shape,y.shape)

(28269, 19) (28269, 12) (21201, 12) (7068, 12) (21201,)


#### Prep for split

In [165]:
train_new = train.drop('Order_No',axis=1)
test_new = test.drop('Order_No',axis=1)
train_new.shape

(21201, 11)

In [168]:
test_new.shape

(7068, 11)

#### train_test_split

In [166]:
X_train, X_test, y_train, y_test = train_test_split(train_new, y, test_size=0.2, random_state = 1, shuffle=False)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16960, 11) (4241, 11) (16960,) (4241,)


### Linear Regression

In [167]:
lr = LinearRegression()
lr.fit(X_train, y_train)
test_pred = lr.predict(X_test)
from sklearn.metrics import mean_squared_error
def rmse(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
rmse(y_test,test_pred)

778.5641538562762

In [169]:
y_pred = lr.predict(test_new)

In [176]:
submission_df = test_df1[['Order No']]
submission_df['Time_Pic_Arr'] = y_pred

In [177]:
submission_df.to_csv('D:/Temp/LRImproved.csv', index = False)

### LR with Scailing

In [172]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_new_scaled = scaler.fit_transform(train_new)
train_new_standardise = pd.DataFrame(train_new_scaled,columns=train_new.columns)
train_new_standardise.head()

,Platform_Type,Distance(km),Temperature,Precipitation(mm),Pickup_Lat,Pickup_Lon,Destination_Lat,Destination_Lon,Day_of_Month,Day_of_Week,Personal_Business
0,0.396406,-0.971370,-0.887117,-0.00314,-1.189424,0.509897,-0.511888,0.414159,-0.756213,1.122927,-0.468583
1,0.396406,1.145469,0.975120,-0.00314,-2.294056,2.349813,-0.356757,0.070177,-0.415254,1.122927,2.134095
2,0.396406,-1.147774,-0.000789,-0.00314,-0.878981,0.858136,-0.526672,0.379588,1.630501,-0.791244,-0.468583
3,0.396406,-0.089354,-1.259565,-0.00314,0.005523,0.563962,0.730374,-0.361289,-0.074294,1.122927,-0.468583
4,-2.802759,-0.089354,-2.438982,-0.00314,0.487515,-0.510932,-0.357811,-0.031369,-0.301601,-1.429301,2.134095


In [173]:
X_train, X_test, y_train, y_test = train_test_split(train_new_standardise, y, test_size=0.2, random_state = 1, shuffle=False)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16960, 11) (4241, 11) (16960,) (4241,)


In [174]:
lr = LinearRegression()
lr.fit(X_train, y_train)
test_pred = lr.predict(X_test)
from sklearn.metrics import mean_squared_error
def rmse(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
rmse(y_test,test_pred)

778.564153856276

In [175]:
y_pred = lr.predict(test_new)

#### Ridge

In [178]:
X_train, X_test, y_train, y_test = train_test_split(train_new_standardise, 
                                                    y, 
                                                    test_size=0.2, 
                                                    shuffle=False)
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(X_train, y_train)
test_pred = lr.predict(X_test)
from sklearn.metrics import mean_squared_error
def rmse(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
rmse(y_test,test_pred)

778.564153856276

In [179]:
y_pred = lr.predict(test_new)

In [180]:
submission_df = test_df1[['Order No']]
submission_df['Time_Pic_Arr'] = y_pred

In [181]:
submission_df.to_csv('D:/Temp/LRRidge.csv', index = False)

#### Lasso

In [184]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train_new_standardise, 
                                                    y, 
                                                    test_size=0.20,
                                                    random_state=1,
                                                    shuffle=False)
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
test_pred = lr.predict(X_test)
def rmse(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
rmse(y_test,test_pred)

778.564153856276